<a href="https://colab.research.google.com/github/Elwing-Chou/ml0804/blob/master/transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
z = zipfile.ZipFile("drive/My Drive/additional/train.zip")
z.extractall("transfer")

In [ ]:
import glob
import pandas as pd
dogs = glob.glob("transfer/train/dog.*")
cats = glob.glob("transfer/train/cat.*")
df = pd.DataFrame({
    "path":dogs + cats,
    "ans":[0] * len(dogs) + [1] * len(cats)
})
df

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg.summary()

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
mlp = [
    BatchNormalization(),
    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.25),
    Dense(2, activation="softmax")
]
for l in vgg.layers:
    l.trainable = False

layers = vgg.layers + mlp
model = Sequential(layers)
model.summary()

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [ ]:
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg16 import preprocess_input

img = load_img(df["path"][0], target_size=(224, 224))
p = preprocess_input(np.array(img))
print(p)
plt.imshow(img)

In [ ]:
# 千萬別直接把df拿來train_test_split
# 只要開始資料處理, 唯一認可numpy array
from sklearn.model_selection import train_test_split
x, y = np.array(df["path"]), np.array(df["ans"])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
print(x_train.shape, x_test.shape)

In [ ]:
def getdata(x, y, batch=20):
    idx = np.random.randint(0, x.shape[0], size=batch)
    paths, ans = x[idx], y[idx]
    ori_imgs, pre_imgs = [], []
    for p in paths:
        img = load_img(p, target_size=(224, 224))
        img_np = np.array(img)
        ori_imgs.append(img_np)
        preprocess = preprocess_input(img_np)
        pre_imgs.append(preprocess)
    return (np.array(ori_imgs), np.array(pre_imgs), np.array(ans))

In [ ]:
for i in range(20):
    print("-" * 15, i, "-" * 15)
    ori, pre, ans = getdata(x_train, y_train)
    result = model.train_on_batch(pre, ans)
    print("[train]:", result)
    ori, pre, ans = getdata(x_test, y_test, 100)
    result = model.test_on_batch(pre, ans)
    print("[validate]:", result)

In [ ]:
# model.predict(pre)
ori, pre, ans = getdata(x_test, y_test, 1000)
model.evaluate(pre, ans)